# In given price Test

In [84]:
import numpy as np
import pandas as pd
import plotly.express as px
from balancerv2cad.StableMath import StableMath
import decimal

## 0. Set parameters of simulation
### 0.1 Contant parameters

In [2]:
# random values
A = 5
balances_start = [3000, 3000, 3000, 3000, 3000]
n = len(balances_start)

### 0.2 Calculate Invariant

In [3]:
D = StableMath.calculateInvariant(A, balances_start)
D

Decimal('15000')

## 1. Test In given out function

### 1.1 Define parameters

In [4]:
index_x = 0
index_y = 1
constant_balances = [balances_start[i] for i in range(len(balances_start)) if i not in [index_x, index_y]]
S = np.sum(constant_balances)
P = np.prod(constant_balances)
print("S:", S)
print("P:", P)

S: 9000
P: 27000000000


### 1.2 Aux funcs

In [5]:
c1 =  (D/(A*(n**n))) + S - D
c2 =  -(D**(n+1)) / (A*(n**(2*n))*P)

def b_func(x):
    return c1+x

def c_func(x):
    return c2/x

def c_derivate_func(x):
    return -c2/(x**2)

def calculate_y(x):
    b = float(b_func(x))
    c = float(c_func(x))
    delta = (b**2) - 4*c
    num = -b + ((delta)**(1/2))
    den = 2
    return num/den

def calcOutGivenIn(amount):
    new_y = calculate_y(balances_start[index_x]+amount)
    return balances_start[index_y] - new_y

### 1.3 Singular test

In [6]:
amount = 100
calculated = calcOutGivenIn(amount)
answer = StableMath.calcOutGivenIn(A, balances_start, index_x, index_y, amount)

Context OUTGIVENIN
Invariant 15000
FinalBalance Out 2900.552908046759877175131169
99.447091953240122824868831
END-CONTEXT OUTGIVENIN


In [7]:
dif = calculated-float(answer)
dif_over_answer = dif*100/float(answer)
print("Difference:", dif)
print(f"Difference over result of BalancerV2Cad: {dif_over_answer}%")

Difference: 0.5518405473616497
Difference over result of BalancerV2Cad: 0.5549086821172451%


### 1.4 Testing with different amount values

In [8]:
print("Calculating...")
results = []
amounts = [1, 50, 100, 200, 500, 1000]
for amount in amounts:
    calculated = calcOutGivenIn(amount)
    answer = StableMath.calcOutGivenIn(A, balances_start, index_x, index_y, amount)
    calculated = calcOutGivenIn(amount)
    answer = StableMath.calcOutGivenIn(A, balances_start, index_x, index_y, amount)
    dif = calculated-float(answer)
    results.append(dif*100/float(answer))

Calculating...
Context OUTGIVENIN
Invariant 15000
FinalBalance Out 2999.000055552473593240704119
0.999944447526406759295881
END-CONTEXT OUTGIVENIN
Context OUTGIVENIN
Invariant 15000
FinalBalance Out 2999.000055552473593240704119
0.999944447526406759295881
END-CONTEXT OUTGIVENIN
Context OUTGIVENIN
Invariant 15000
FinalBalance Out 2950.138530728993140625124319
49.861469271006859374875681
END-CONTEXT OUTGIVENIN
Context OUTGIVENIN
Invariant 15000
FinalBalance Out 2950.138530728993140625124319
49.861469271006859374875681
END-CONTEXT OUTGIVENIN
Context OUTGIVENIN
Invariant 15000
FinalBalance Out 2900.552908046759877175131169
99.447091953240122824868831
END-CONTEXT OUTGIVENIN
Context OUTGIVENIN
Invariant 15000
FinalBalance Out 2900.552908046759877175131169
99.447091953240122824868831
END-CONTEXT OUTGIVENIN
Context OUTGIVENIN
Invariant 15000
FinalBalance Out 2802.204453054920192068427095
197.795546945079807931572905
END-CONTEXT OUTGIVENIN
Context OUTGIVENIN
Invariant 15000
FinalBalance Out 280

In [9]:
print("Out Given In simulation")
print("Difference of the our function result from the BalancerV2Cad result over the result of BalancerV2Cad package")
print("----------------------------------------------")
for r, amount in zip(results, amounts):
    amount_over_balance = amount * 100 / balances_start[index_x]
    print(f"Input of {amount} token, or {amount_over_balance:.2f}% of the amount of tokens in the pool, got error of {r:.2f}%")

Out Given In simulation
Difference of the our function result from the BalancerV2Cad result over the result of BalancerV2Cad package
----------------------------------------------
Input of 1 token, or 0.03% of the amount of tokens in the pool, got error of 0.01%
Input of 50 token, or 1.67% of the amount of tokens in the pool, got error of 0.28%
Input of 100 token, or 3.33% of the amount of tokens in the pool, got error of 0.55%
Input of 200 token, or 6.67% of the amount of tokens in the pool, got error of 1.11%
Input of 500 token, or 16.67% of the amount of tokens in the pool, got error of 2.83%
Input of 1000 token, or 33.33% of the amount of tokens in the pool, got error of 5.97%


## 2. Spot price given in test

### 2.1 Define aux functions

In [68]:
def calculate_spotprice(x):
    b = float(b_func(x))
    c = float(c_func(x))
    dc = float(c_derivate_func(x))
    num = b-(2*dc)
    in_root = (b**2) - 4*c
    den = 2*(in_root**(1/2))
    return (num/den) - 1/2

def calcSpotPriceGivenIn(amount):
    new_spot_price = calculate_price(balances_start[index_x]+amount)
    return -new_spot_price

def simulateSpotPrice(amount, small_trade):
    new_balances = balances_start[:]
    print('-------previous-trade--------')
    out = float(StableMath.calcOutGivenIn(A, balances_start, index_x, index_y, amount))
    new_balances[index_x]+=amount
    new_balances[index_y]-=out
    print(new_balances)
    print('-------small-trade--------')
    return StableMath.calcOutGivenIn(A, new_balances, index_x, index_y, small_trade)/small_trade

### 2.2 Singular test

In [75]:
amount = 50
calculated = calcSpotPriceGivenIn(amount)
answer = simulateSpotPrice(amount, 1)

-------previous-trade--------
Context OUTGIVENIN
Invariant 15000
FinalBalance Out 2950.138530728993140625124319
49.861469271006859374875681
END-CONTEXT OUTGIVENIN
[3050, Decimal('2950.138530728993140625124319'), 3000, 3000, 3000]
-------small-trade--------
Context OUTGIVENIN
Invariant 15001
FinalBalance Out 2950.141284906568104161241438
-0.002754177574963536117119
END-CONTEXT OUTGIVENIN


In [76]:
print(calculated)
print(answer)

0.9999893309082654
-0.002754177574963536117119


### 2.3 Iterate over diferent sizes of "small-trade"

In [94]:
print("Calculating...")
results = pd.DataFrame(columns=["small trade", "amount", "calculated result", "simulated result"])
smalltrades = [decimal.Decimal(0.1), 1, 2, 6, 10]
amounts = [1, 10, 50, 100, 500, 1000]

for s in smalltrades:
    for a in amounts:
        calc = calcSpotPriceGivenIn(a)
        simulated = float(simulateSpotPrice(a, s))
        new_row = {"small trade": s, "amount": a, "calculated result": calc, "simulated result":simulated}
        results = results.append(new_row, ignore_index=True)

Calculating...
-------previous-trade--------
Context OUTGIVENIN
Invariant 15000
FinalBalance Out 2999.000055552473593240704119
0.999944447526406759295881
END-CONTEXT OUTGIVENIN
[3001, Decimal('2999.000055552473593240704119'), 3000, 3000, 3000]
-------small-trade--------
Context OUTGIVENIN
Invariant 15001
FinalBalance Out 2999.900028326789425686857274
-0.899972774315832446153155
END-CONTEXT OUTGIVENIN
-------previous-trade--------
Context OUTGIVENIN
Invariant 15000
FinalBalance Out 2990.005552513645367121314825
9.994447486354632878685175
END-CONTEXT OUTGIVENIN
[3010, Decimal('2990.005552513645367121314825'), 3000, 3000, 3000]
-------small-trade--------
Context OUTGIVENIN
Invariant 15001
FinalBalance Out 2990.905124357099122911887305
-0.899571843453755790572480
END-CONTEXT OUTGIVENIN
-------previous-trade--------
Context OUTGIVENIN
Invariant 15000
FinalBalance Out 2950.138530728993140625124319
49.861469271006859374875681
END-CONTEXT OUTGIVENIN
[3050, Decimal('2950.13853072899314062512431

/tmp/ipykernel_162/661080488.py:11: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_162/661080488.py:11: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_162/661080488.py:11: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_162/661080488.py:11: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_162/661080488.py:11: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_162/661080488.py:11: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

In [101]:
fig = px.scatter(results, x="small trade", y="simulated result")
fig.show()

In [100]:
fig = px.scatter(results, x="amount", y="calculated result")
fig.show()

# 3. In given Price test

## 3.1 Aux funcs

In [117]:
def f6(dy):
    return 4*(dy**2)+(4*dy)

def f5(dy):
    return (8*float(c1)*(dy**2))+(8*float(c1)*dy)

def f4(dy):
    return (4*(float(c1)**2)*(dy**2))+(4*(float(c1)**2)*dy)

def f3(dy):
    return (-16*float(c2)*(dy**2)) - (16*float(c2)*dy) - 8*float(c2)

def f2(dy):
    return -4*float(c1)*float(c2)

def f1(dy):
    return 0

def f0(dy):
    return -4*(float(c2)**2)


def calculate_x(price):
    spot_price = -price
    params = [f(spot_price) for f in [f6, f5, f4, f3, f2, f1, f0]]
    return np.roots(params)

def calcInGivenPrice(price):
    roots = calculate_x(price)
    initial_x = balances_start[index_x]
    possible_results = []
    for r in roots:
        if r.imag == 0:
            if r.real > initial_x:
                possible_results.append(r.real)
    return min(possible_results) - initial_x

### 3.2 Unique test

In [122]:
calcInGivenPrice(0.9)

2898.352618085748

In [123]:
calcSpotPriceGivenIn(decimal.Decimal('2898.352618085748'))

0.8999999999999709

### 3.3 Test for different results

In [129]:
prices= [0.98, 0.95, 0.9, 0.8, 0.7]
for p in prices:
    in_amount = calcInGivenPrice(p)
    price = calcSpotPriceGivenIn(decimal.Decimal(str(in_amount)))
    print(f'To price of {p} error is {abs(p-price)*100/p}%')

To price of 0.98 error is 2.6396118850781785e-12%
To price of 0.95 error is 6.637965031443041e-12%
To price of 0.9 error is 3.2319825827976777e-12%
To price of 0.8 error is 6.544764730165298e-11%
To price of 0.7 error is 6.693058805597373e-12%
